In [1]:
# Import Libraries
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from pymongo import MongoClient
from dotenv import load_dotenv
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import requests

In [2]:
# Add the 'scripts' directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

In [3]:
import extract_to_mongodb as etm
import db_utils as dbu

In [ ]:
# print(dir(dbu))

In [4]:
collection_name = os.getenv('COLLECTION_NAME_FEATUREENGINEERED')
naturalearth_lowres = os.getenv('NATURALEARTH_SHAPEFILE_PATH')

In [5]:
print(f"Collection Name: {collection_name}")

Collection Name: wildfire_feature_engineered_data


Load the Data

In [7]:
# Load the cleaned data
geo_wfp = dbu.load_all_data_from_mongodb(collection_name)

In [8]:
geo_wfp.head(5)

,_id,temp,rh,ws,wd,pcp,ffmc,dmc,dc,isi,...,tfc0,sfc0,year,month,day,lat_sin,lat_cos,lon_sin,lon_cos,year_month
0,66846cd755d3554c96b02c66,-1.006741,2.094934,-0.649149,320,0.430,82.976,30.078,161.161,2.680,...,0.35,0.35,2020,6,2,0.883899,0.467678,-0.311904,-0.950114,2020-6
1,66846cd755d3554c96b02c67,0.423696,0.992790,-0.700308,145,1.237,68.466,0.000,294.020,0.977,...,0.10,0.10,2020,6,11,0.879890,0.475177,-0.268096,-0.963392,2020-6
2,66846cd755d3554c96b02c68,0.226618,1.122454,-0.867970,30,0.591,88.685,55.743,202.448,5.536,...,1.36,1.36,2020,6,20,0.736971,0.675925,-0.939322,0.343037,2020-6
3,66846cd755d3554c96b02c69,1.366794,-1.665321,-0.843150,271,0.000,98.652,290.568,841.230,22.181,...,0.35,0.35,2020,6,22,0.535709,0.844403,-0.934801,-0.355172,2020-6
4,66846cd755d3554c96b02c6a,1.170139,0.020311,-0.000958,50,0.001,91.660,18.664,102.620,10.972,...,0.35,0.35,2020,6,13,0.551529,0.834155,-0.999729,-0.023267,2020-6


In [9]:
app_name = os.getenv('APP_NAME')
contact_email = os.getenv('CONTACT_EMAIL')

In [10]:
def reverse_geocode_nominatim(lat, lon):
    url = 'https://nominatim.openstreetmap.org/reverse'
    params = {
        'format': 'json',
        'lat': lat,
        'lon': lon,
        'zoom': 10,
        'addressdetails': 1
    }
    headers = {
        'User-Agent': f'{app_name} ({contact_email})'
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        address = data.get('address', {})
        city = address.get('city', None)
        if not city:
            city = address.get('town', None)
        if not city:
            city = address.get('village', None)
        if not city:
            city = address.get('county', None)
        return city
    else:
        print(f"Error: {response.status_code}")
        return None

In [11]:
# Calculate latitude and longitude from sine and cosine values
def calculate_lat_lon(row):
    lat = np.arcsin(row.lat_sin) * 180 / np.pi
    lon = np.arctan2(row.lon_sin, row.lon_cos) * 180 / np.pi
    return lat, lon




In [12]:
# Apply reverse geocoding to the first 5 records using computed latitude and longitude
df_first_5 = df.iloc[:5].copy()
df_first_5['lat'], df_first_5['lon'] = zip(*df_first_5.apply(calculate_lat_lon, axis=1))
df_first_5['city'], df_first_5['country'] = zip(*df_first_5.apply(lambda row: reverse_geocode_nominatim(row.lat, row.lon), axis=1))

# Print the first 5 records with the city and country columns
print(df_first_5[['lat', 'lon', 'city', 'country']])

       lat       lon                            city
0  62.1163 -161.8260            Kusilvak Census Area
1  61.6291 -164.4490            Kusilvak Census Area
2  47.4740  -69.9380  Saint-Denis-De La Bouteillerie
3  32.3920 -110.8040                     Pima County
4  33.4720  -91.3332                   Chicot County


In [13]:
# # Apply reverse geocoding to the first 5 records using longitude and latitude
# geo_wfp_first_5 = geo_wfp.iloc[:5].copy()
# geo_wfp_first_5['city'] = geo_wfp_first_5.apply(lambda row: reverse_geocode_nominatim(row.lat, row.lon), axis=1)

# # Merge the city data back into the original DataFrame
# geo_wfp = geo_wfp.merge(geo_wfp_first_5[['city']], left_index=True, right_index=True, how='left')

# Apply reverse geocoding to all records using computed latitude and longitude
geo_wfp['lat'], geo_wfp['lon'] = zip(*geo_wfp.apply(calculate_lat_lon, axis=1))
geo_wfp['city'] = geo_wfp.apply(lambda row: reverse_geocode_nominatim(row.lat, row.lon), axis=1)


KeyboardInterrupt: 

In [14]:
geo_wfp.head(5)

,_id,temp,rh,ws,wd,pcp,ffmc,dmc,dc,isi,...,year,month,day,lat_sin,lat_cos,lon_sin,lon_cos,year_month,lat,lon
0,66846cd755d3554c96b02c66,-1.006741,2.094934,-0.649149,320,0.430,82.976,30.078,161.161,2.680,...,2020,6,2,0.883899,0.467678,-0.311904,-0.950114,2020-6,62.1163,-161.8260
1,66846cd755d3554c96b02c67,0.423696,0.992790,-0.700308,145,1.237,68.466,0.000,294.020,0.977,...,2020,6,11,0.879890,0.475177,-0.268096,-0.963392,2020-6,61.6291,-164.4490
2,66846cd755d3554c96b02c68,0.226618,1.122454,-0.867970,30,0.591,88.685,55.743,202.448,5.536,...,2020,6,20,0.736971,0.675925,-0.939322,0.343037,2020-6,47.4740,-69.9380
3,66846cd755d3554c96b02c69,1.366794,-1.665321,-0.843150,271,0.000,98.652,290.568,841.230,22.181,...,2020,6,22,0.535709,0.844403,-0.934801,-0.355172,2020-6,32.3920,-110.8040
4,66846cd755d3554c96b02c6a,1.170139,0.020311,-0.000958,50,0.001,91.660,18.664,102.620,10.972,...,2020,6,13,0.551529,0.834155,-0.999729,-0.023267,2020-6,33.4720,-91.3332


In [ ]:
geo_wfp.drop(columns=['lon, lat'], inplace=True)

In [ ]:
#analysis only, this part can be removed
#geo_wfp.to_csv('engineered_wildfire_data.csv', index=False)

In [ ]:
# Save the csv featured engineering data to mongodb
#dbu.insert_data_to_mongodb('engineered_wildfire_data.csv', os.getenv('COLLECTION_NAME_FEATUREENGINEERED'))


In [ ]:
# Save the dataframe -  featured engineering data to mongodb
dbu.insert_df_only_to_mongodb(geo_wfp, os.getenv('COLLECTION_NAME_FEATUREENGINEERED_WITH_CITY'))